In [131]:
import pandas as pd
import numpy as np
import random
import json
from enum import Enum
from pathlib import Path

In [127]:
df = pd.read_csv("data/campusplan_evaluation.csv")

In [128]:
# fake building names
fake_lecture_halls = ["Einstein-Hörsaal", "Curie-Hörsaal", "Newton-Hörsaal", "Hawking-Hörsaal", "Galileo-Hörsaal", "Darwin-Hörsaal", "Tesla-Hörsaal", "Feynman-Hörsaal", "Planck-Hörsaal", "Bohr-Hörsaal", "Faraday-Hörsaal", "Mendel-Hörsaal", "Hubble-Hörsaal", "Turing-Hörsaal", "Rutherford-Hörsaal", "Archimedes-Hörsaal", "Sanger-Hörsaal", "von Neumann-Hörsaal", "Franklin-Hörsaal"]
fake_buildings = ["60.03", "60.20", "60.41", "60.19", "60.12", "20.61", "20.65", "20.91", "10.14", "10.74", "10.28", "10.35", "30.09", "30.26", "30.52", "40.20", "40.35", "40.61", "50.09", "50.78", "70.02"]

In [134]:
class Turns(Enum):
    SINGLE = 1
    MULTI = 2
    
def asemble_prompt(input_data, response_data, prompt, response):
    prompt = {"input_data" : input_data, "response_data" : response_data, "prompt" : prompt, "response" : response}
    return prompt

def build_prompt_dict(turns, prompts):
    prompt_dict = {"num_turns" : turns, "prompts" : prompts}
    return prompt_dict

def json_dump(turns, prompts, name):
    if turns == Turns.SINGLE:
        path = Path("data/evaluation/single_turn")
    elif turns == Turns.MULTI:
        path = Path("data/evaluation/multi_turn")

    with open(path / f"{name}.json", 'w') as f:
        json.dump(prompts, f, indent=4)

## Retrieve Static Information

In [141]:
# building location
prompts = []
for i in df.iterrows():
    index = random.randint(0, df.shape[0]-1)
    building_id = df['title'].loc[index]
    address = df['rev_display_name'].astype(str).loc[index]
    
    prompt = f"Wo ist Gebäude {building_id}?"
    
    if address == "nan":
        response = "Ich kenne die Adresse von diesem Gebäude leider nicht."
    else:
        response = f"Das Gebäude hat die Adresse {address}"

    prompt = asemble_prompt(building_id, address, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_location")

# fake building location
prompts = []
for fake_hall in fake_lecture_halls:
    prompt = f"Wo ist Gebäude {fake_hall}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_hall, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)

for fake_building in fake_buildings:
    prompt = f"Wo ist Gebäude {fake_building}?"
    response = "Zu diesem Gebäude habe ich leider keine Informationen."

    prompt = asemble_prompt(fake_building, None, prompt, response)
    prompt_dict = build_prompt_dict(1, [prompt])
    prompts.append(prompt_dict)
    
json_dump(Turns.SINGLE, prompts, "building_location_fake")